In [9]:
import ctypes, inspect, os, graphlab
from ctypes import wintypes
kernel32 = ctypes.WinDLL('kernel32', use_last_error=True)
kernel32.SetDllDirectoryW.argtypes = (wintypes.LPCWSTR,)
src_dir = os.path.split(inspect.getfile(graphlab))[0]
kernel32.SetDllDirectoryW(src_dir)

1

In [10]:
#load the data
products = graphlab.SFrame('amazon_baby.gl/')

In [13]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [14]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ..."


In [15]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

graphlab.canvas.set_target("ipynb")
def countWords(s):
    dict=products["word_count"]
    if (s in dict):
        return products['word_count'][s]
    else:
        return 0
    
for w in selected_words:
    products[w]=products['word_count'].apply(lambda x:x[w] if w in x else 0)
    print 'word %s sum is : %s' % (w, products[w].sum())


RuntimeError: Runtime Exception. Unable to evaluate lambdas. Lambda workers did not start.

In [5]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ..."


In [7]:
train_data,test_data = products.random_split(.8, seed=0)
test_data['sentiment'].show('Categorical')

RuntimeError: Runtime Exception. Column name sentiment does not exist.

In [8]:
sentiment_model = graphlab.logistic_classifier.create(train_data, target='sentiment', features=['word_count'], 
                                                      validation_set=test_data)
print sentiment_model['coefficients'].sort('value', ascending=False)

selected_words_model = graphlab.logistic_classifier.create(train_data, target='sentiment', features=selected_words, 
                                                           validation_set=test_data)
print selected_words_model['coefficients'].sort('value', ascending=False)

print 'evaluation.....'
sentiment_model.evaluate(test_data, metric='roc_curve')
sentiment_model.show(view='Evaluation')

ToolkitError: The following columns were expected but are missing: ['sentiment']

In [ ]:
print " understanding the review..."
# get the data for another product: Baby Trend Diaper Champ
baby_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

baby_champ_reviews['predicted_sentiment'] = selected_words_model.predict(baby_champ_reviews, output_type='probability')
baby_champ_reviews = baby_champ_reviews.sort('rating', ascending=False)
print selected_words_model.predict(baby_champ_reviews[0:1], output_type='probability')

baby_champ_reviews['predicted_sentiment'] = sentiment_model.predict(baby_champ_reviews, output_type='probability')
baby_champ_reviews = baby_champ_reviews.sort('rating', ascending=False)
print sentiment_model.predict(baby_champ_reviews[0:1], output_type='probability')

for word in selected_words:
    print baby_champ_reviews[0][word]